In [1]:
############################# Importing all req. libreries ############################################
from pymongo import MongoClient
import pandas as pd
from mpl_toolkits.basemap import Basemap
import matplotlib
%matplotlib qt
import matplotlib.pyplot as plt
import ipywidgets as ipw
from ipywidgets import Button, HBox, VBox,RadioButtons
client = MongoClient("mongodb://localhost:27017")
database = client["Open_Source_DB"]


In [11]:
collection = database["temp"]
collection.insert({"location" : {
        "type" : "Point", 
        "coordinates" : [
            13.4, 
            52.516667000000005
        ]
    }})
collection.insert({"location" : {
        "type" : "Point", 
        "coordinates" : [
            9.180819, 
            48.782343
        ]
    }})

C:\Users\anilk\Anaconda3\envs\Final\lib\site-packages\ipykernel_launcher.py:6: DeprecationWarning: insert is deprecated. Use insert_one or insert_many instead.
  
C:\Users\anilk\Anaconda3\envs\Final\lib\site-packages\ipykernel_launcher.py:13: DeprecationWarning: insert is deprecated. Use insert_one or insert_many instead.
  del sys.path[0]


ObjectId('5e5856a421dceebcfbd8a937')

In [16]:
#set in shell
#db.temp.createIndex( { location : "2dsphere" } )

In [15]:
############################# MongoDB Pipeline  ############################################

pipeline = [
    {
        u"$geoNear": {
            u"near": {
                u"type": u"Point",
                u"coordinates": [
                    9.180819, 
                    48.782343
                ]
            },
            u"distanceField": u"dist.calculated",
        }
    }, 
    {
        u"$project": {
            u"Entitiy": u"$Type of User",
            u"coordinates": u"$location.coordinates",
            u"_id": 0.0
        }
    }
]

cursor = collection.aggregate(
    pipeline, 
    allowDiskUse = False
)

lats = []
lons = []
ty = []
try:
    for doc in cursor:
        print(doc)
#         lats.append(doc["coordinates"][1])
#         lons.append(doc["coordinates"][0])
#         ty.append(doc["Entitiy"])
finally:
    client.close()


{'coordinates': [9.180819, 48.782343]}
{'coordinates': [13.4, 52.516667000000005]}


In [69]:
####################### Visulizing the results in Map ###################################################
m = Basemap(projection='mill',
                resolution='l') 

m.drawcoastlines()
m.drawcountries(linewidth=0.1)
m.drawstates()

i = 0
for lat , lon in zip(lats,lons):
    x,y = m.projtran(lon,lat)
    if ty[i] == 'End User' : m.plot(x,y,'g*',markersize=10)
    if ty[i] == 'Retailer' : m.plot(x,y,'y*',markersize=10)
    if ty[i] == 'Supplier' : m.plot(x,y,'b*',markersize=10)
    if ty[i] == 'Manufacturer' : m.plot(x,y,'b*',markersize=10)
    i = i+1
plt.show()

C:\Users\anilk\Anaconda3\envs\Final\lib\site-packages\ipykernel_launcher.py:3: MatplotlibDeprecationWarning: 
The dedent function was deprecated in Matplotlib 3.1 and will be removed in 3.3. Use inspect.cleandoc instead.
  This is separate from the ipykernel package so we can avoid doing imports until
C:\Users\anilk\Anaconda3\envs\Final\lib\site-packages\ipykernel_launcher.py:6: MatplotlibDeprecationWarning: 
The dedent function was deprecated in Matplotlib 3.1 and will be removed in 3.3. Use inspect.cleandoc instead.
  
C:\Users\anilk\Anaconda3\envs\Final\lib\site-packages\ipykernel_launcher.py:7: MatplotlibDeprecationWarning: 
The dedent function was deprecated in Matplotlib 3.1 and will be removed in 3.3. Use inspect.cleandoc instead.
  import sys


In [57]:
?zip